In [3]:
import pandas as pd
import panel as pn
import psycopg2
from sqlalchemy import create_engine
from data.database import conectar_postgres

pn.extension()

# Função para pesquisar clientes no PostgreSQL com base em um filtro
def pesquisar_clientes(filtro_coluna, filtro_valor):
    conn = conectar_postgres()
    if conn:
        engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)
        query = f"SELECT * FROM Clientes WHERE {filtro_coluna} = '{filtro_valor}'"
        result_df = pd.read_sql_query(query, engine)
        conn.close()
        return result_df if result_df is not None else pd.DataFrame()
    return pd.DataFrame()

# Funções CRUD para clientes
def consultar_clientes():
    conn = conectar_postgres()
    if conn:
        engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)
        query = "SELECT * FROM Clientes"
        result_df = pd.read_sql_query(query, engine)
        conn.close()
        return result_df if result_df is not None else pd.DataFrame()
    return pd.DataFrame()

def inserir_cliente(nome, sobrenome, apelido, telefone1, telefone2, rua, numero, cidade, estado):
    conn = conectar_postgres()
    if conn:
        with conn.cursor() as cursor:
            cursor.execute(
                "INSERT INTO Clientes (nome, sobrenome, apelido, telefone1, telefone2, rua, numero, cidade, estado) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                (nome, sobrenome, apelido, telefone1, telefone2, rua, numero, cidade, estado))
            conn.commit()
        conn.close()

def excluir_cliente(cliente_id):
    # Remover registros relacionados na tabela "senhas_numeros"
    conn = conectar_postgres()
    if conn:
        try:
            with conn.cursor() as cursor:
                # Remover registros relacionados na tabela "senhas_numeros"
                cursor.execute("DELETE FROM senhas_numeros WHERE id_cliente = %s", (cliente_id,))
                conn.commit()

            # Excluir o cliente
            with conn.cursor() as cursor:
                cursor.execute("DELETE FROM Clientes WHERE id_cliente = %s", (cliente_id,))
                conn.commit()

        except psycopg2.Error as e:
            print(f"Erro ao excluir cliente: {e}")
        finally:
            conn.close()

# Widget para exibir clientes
clientes_output = pn.widgets.DataFrame()

# Widgets de entrada
nome_input = pn.widgets.TextInput(name="Nome")
sobrenome_input = pn.widgets.TextInput(name="Sobrenome")
apelido_input = pn.widgets.TextInput(name="Apelido")
telefone1_input = pn.widgets.TextInput(name="Telefone 1")
telefone2_input = pn.widgets.TextInput(name="Telefone 2")
rua_input = pn.widgets.TextInput(name="Rua")
numero_input = pn.widgets.IntInput(name="Número")
cidade_input = pn.widgets.TextInput(name="Cidade")
estado_input = pn.widgets.TextInput(name="Estado")
id_cliente_input = pn.widgets.TextInput(name="ID do Cliente (para atualização/exclusão)")

# Campo de entrada para o valor de pesquisa
filtro_valor_input = pn.widgets.TextInput(name="Valor de Pesquisa")

# Botão de Atualizar
atualizar_button = pn.widgets.Button(name="Atualizar")

def atualizar_cliente(cliente_id, **kwargs):
    # Atualizar os campos do cliente no banco de dados
    conn = conectar_postgres()
    if conn:
        with conn.cursor() as cursor:
            set_clause = ', '.join([f"{key} = %s" for key in kwargs.keys()])
            update_query = f"UPDATE Clientes SET {set_clause} WHERE id_cliente = %s"
            cursor.execute(update_query, list(kwargs.values()) + [cliente_id])
            conn.commit()
        conn.close()


# Função para atualizar os campos modificados
def on_atualizar_button_click(event):
    cliente_id = int(id_cliente_input.value)
    nome = nome_input.value
    sobrenome = sobrenome_input.value
    apelido = apelido_input.value
    telefone1 = telefone1_input.value
    telefone2 = telefone2_input.value
    rua = rua_input.value
    numero = numero_input.value
    cidade = cidade_input.value
    estado = estado_input.value

    if nome:
        atualizar_cliente(cliente_id, nome=nome)
    if sobrenome:
        atualizar_cliente(cliente_id, sobrenome=sobrenome)
    if apelido:
        atualizar_cliente(cliente_id, apelido=apelido)
    if telefone1:
        atualizar_cliente(cliente_id, telefone1=telefone1)
    if telefone2:
        atualizar_cliente(cliente_id, telefone2=telefone2)
    if rua:
        atualizar_cliente(cliente_id, rua=rua)
    if numero:
        atualizar_cliente(cliente_id, numero=numero)
    if cidade:
        atualizar_cliente(cliente_id, cidade=cidade)
    if estado:
        atualizar_cliente(cliente_id, estado=estado)

    on_exibir_button_click(None)

# Botão de Inserir
inserir_button = pn.widgets.Button(name="Inserir")

# Função para inserir um novo cliente
def on_inserir_button_click(event):
    nome = nome_input.value
    sobrenome = sobrenome_input.value
    apelido = apelido_input.value
    telefone1 = telefone1_input.value
    telefone2 = telefone2_input.value
    rua = rua_input.value
    numero = numero_input.value
    cidade = cidade_input.value
    estado = estado_input.value

    # Inserir o novo cliente no banco de dados
    inserir_cliente(nome, sobrenome, apelido, telefone1, telefone2, rua, numero, cidade, estado)

    # Limpar os campos de entrada
    nome_input.value = ''
    sobrenome_input.value = ''
    apelido_input.value = ''
    telefone1_input.value = ''
    telefone2_input.value = ''
    rua_input.value = ''
    numero_input.value = None
    cidade_input.value = ''
    estado_input.value = ''

    # Atualizar a exibição dos clientes
    on_exibir_button_click(None)

# Conectar função ao evento de clique do botão de Inserir
inserir_button.on_click(on_inserir_button_click)

# Botão de Excluir
excluir_button = pn.widgets.Button(name="Excluir")

# Função para excluir um cliente
def on_excluir_button_click(event):
    cliente_id = int(id_cliente_input.value)
    excluir_cliente(cliente_id)
    on_exibir_button_click(None)

# Botão de Pesquisar
pesquisar_button = pn.widgets.Button(name="Pesquisar")

# Função para pesquisar clientes
def on_pesquisar_button_click(event):
    filtro_coluna = filtro_coluna_dropdown.value
    filtro_valor = filtro_valor_input.value
    result_df = pesquisar_clientes(filtro_coluna, filtro_valor)
    if result_df is not None:
        clientes_output.value = result_df

# Botão de Exibir/Ocultar Clientes
exibir_button = pn.widgets.Button(name="Exibir Clientes")

# Função para exibir/ocultar clientes
def on_exibir_button_click(event):
    result_df = consultar_clientes()
    if result_df is not None:
        clientes_output.value = result_df
        clientes_output.visible = not clientes_output.visible

# Dropdown para seleção do campo de filtro
filtro_coluna_dropdown = pn.widgets.Select(options=["nome", "sobrenome", "apelido", "telefone1", "telefone2", "rua", "numero", "cidade", "estado"], name="Filtrar por")

# Conectar funções aos eventos dos botões
atualizar_button.on_click(on_atualizar_button_click)
excluir_button.on_click(on_excluir_button_click)
pesquisar_button.on_click(on_pesquisar_button_click)
exibir_button.on_click(on_exibir_button_click)

# Montar layout
layout = pn.Column(
    "## CRUD de Clientes",
    nome_input,
    sobrenome_input,
    apelido_input,
    telefone1_input,
    telefone2_input,
    rua_input,
    numero_input,
    cidade_input,
    estado_input,
    inserir_button,
    pn.Row(id_cliente_input, excluir_button,atualizar_button,),
    pn.Row(filtro_valor_input, filtro_coluna_dropdown, pesquisar_button),
    pn.Row(clientes_output, exibir_button)
)

# Chamar a função para exibir os clientes ao criar o layout
on_exibir_button_click(None)

# Exibir layout
layout


Column
    [0] Markdown(str)
    [1] TextInput(name='Nome')
    [2] TextInput(name='Sobrenome')
    [3] TextInput(name='Apelido')
    [4] TextInput(name='Telefone 1')
    [5] TextInput(name='Telefone 2')
    [6] TextInput(name='Rua')
    [7] IntInput(name='Número')
    [8] TextInput(name='Cidade')
    [9] TextInput(name='Estado')
    [10] Button(name='Inserir')
    [11] Row
        [0] TextInput(name='ID do Cliente (...)
        [1] Button(name='Excluir')
        [2] Button(name='Atualizar')
    [12] Row
        [0] TextInput(name='Valor de Pesquisa')
        [1] Select(name='Filtrar por', options=['nome', 'sobrenome', ...], value='nome')
        [2] Button(name='Pesquisar')
    [13] Row
        [0] DataFrame(value=    id_cliente  ..., visible=False)
        [1] Button(name='Exibir Clientes')